In [48]:
'''
This code will grab USGS streamflow data for a defined period of interest 

'''

# first import the functions for downloading data from NWIS
import dataretrieval.nwis as nwis
import pandas as pd

# specify the USGS site code for which we want data.
site = ['01054500']

# get instantaneous values (iv)
df = nwis.get_record(sites=site, service='iv', start='2017-12-01', end='2018-01-01')
df.reset_index(inplace=True) #reset index to grab station number and date
df_quarter = df.iloc[:,0:2]
print(df_quarter)

df_copy = df.copy()
#print(df_copy)
df_copy['NewDateTime'] = pd.to_datetime(df_copy['datetime'])
df_copy.set_index("NewDateTime",inplace=True)
df_copy.reset_index(inplace=True) #reset indexes so can grab "Date" column
df_copy.drop("datetime", axis=1, inplace=True)
#print(df_copy)
df_copy.index = df_copy['NewDateTime'] # index so can pull date time in resample
df_avg = df_copy.resample('H').mean()
df_avg.reset_index(inplace=True) 
df_avgflow = df_avg.iloc[:,0:2]
print(df_avgflow)

# start BMI USGS 

                      datetime   00060
0    2017-12-01 00:00:00-05:00  3370.0
1    2017-12-01 00:15:00-05:00  3350.0
2    2017-12-01 00:30:00-05:00  3350.0
3    2017-12-01 00:45:00-05:00  3350.0
4    2017-12-01 01:00:00-05:00  3350.0
...                        ...     ...
3067 2018-01-01 22:45:00-05:00  2800.0
3068 2018-01-01 23:00:00-05:00  2820.0
3069 2018-01-01 23:15:00-05:00  2800.0
3070 2018-01-01 23:30:00-05:00  2820.0
3071 2018-01-01 23:45:00-05:00  2820.0

[3072 rows x 2 columns]
                            00060   00065
NewDateTime                              
2017-12-01 00:00:00-05:00  3355.0  4.3825
2017-12-01 01:00:00-05:00  3350.0  4.3800
2017-12-01 02:00:00-05:00  3355.0  4.3825
2017-12-01 03:00:00-05:00  3350.0  4.3800
2017-12-01 04:00:00-05:00  3355.0  4.3825
...                           ...     ...
2018-01-01 19:00:00-05:00  2760.0  4.1000
2018-01-01 20:00:00-05:00  2785.0  4.1125
2018-01-01 21:00:00-05:00  2790.0  4.1150
2018-01-01 22:00:00-05:00  2800.0  4.1200
201

In [16]:
df

00060 00060_cd  00065 00065_cd
site_no  datetime                                                  
01054500 2017-12-01 00:00:00-05:00  3370.0        A   4.39        A
         2017-12-01 00:15:00-05:00  3350.0        A   4.38        A
         2017-12-01 00:30:00-05:00  3350.0        A   4.38        A
         2017-12-01 00:45:00-05:00  3350.0        A   4.38        A
         2017-12-01 01:00:00-05:00  3350.0        A   4.38        A
...                                    ...      ...    ...      ...
01055000 2018-01-01 22:45:00-05:00     NaN      NaN   1.61        A
         2018-01-01 23:00:00-05:00     NaN      NaN   1.61        A
         2018-01-01 23:15:00-05:00     NaN      NaN   1.61        A
         2018-01-01 23:30:00-05:00     NaN      NaN   1.61        A
         2018-01-01 23:45:00-05:00     NaN      NaN   1.61        A

[6144 rows x 4 columns]